In [2]:
from bs4 import BeautifulSoup
import requests

BASE_URL = "https://en.wikipedia.org/wiki/List_of_accidents_and_incidents_involving_commercial_aircraft"

In [3]:
response = requests.get(BASE_URL)

if response.status_code==200:
    html_content = response.text
else:
    print("Status code - ", response.status_code)

In [4]:
soup = BeautifulSoup(html_content, 'html.parser')

## Extract 

In [5]:
def extract_data():
    def extract_years():
        level_one = soup.find('div', class_='mw-page-container').find('div', class_='mw-page-container-inner').find('div', class_='mw-content-container')

        year_by_year = level_one.find('div', id='bodyContent').find('div', id='mw-content-text').find_all('div', class_='mw-heading mw-heading3')

        all_years = []
        for year in year_by_year:
            all_years.append(year.find('h3').text)
        
        return all_years


    def extrtact_content():
        import re
        parent_div = soup.find('div', id='bodyContent').find('div', id='mw-content-text').find('div', class_='mw-content-ltr mw-parser-output')
        all_uld = parent_div.find_all(recursive=False)


        all_content = []

        regex = r'^(January|February|March|April|May|June|July|August|September|October|November|December)\s\d+'

        for element in parent_div.children:
            temp = []

            # Process only <ul> elements
            if element.name == 'ul':
                # Extract <li> elements from <ul>
                for li in element.find_all('li', recursive=False):
                    text = li.text
                    if re.search(regex, text):  # Check if it matches a date format
                        temp.append(text)


            # Append to the final list if there is any content found
            if temp:
                all_content.append(temp)
        
        return all_content
    
    all_years = extract_years()
    all_content = extrtact_content()

    return all_years, all_content

all_years, all_content = extract_data()
print(len(all_years), len(all_content))


105 153


## Additional Cleaning of data done Manually

In [6]:
merge_indices = [
    [29, 30],
    [32, 33],
    [36, 37],
    [38, 39, 40],
    [42, 43],
    [45, 46],
    [47, 48],
    [49, 50],
    [51, 52],
    [55, 56],
    [61, 62],
    [63, 64, 65],
    [66, 67, 68],
    [80, 81],
    [86, 87],
    [88, 89],
    [91, 92],
    [93, 94],
    [95, 96],
    [97, 98, 99],
    [105, 106],
    [107, 108, 109],
    [117, 118, 119],
    [121, 122, 123],
    [125, 126],
    [127, 128, 129, 130, 131],
    [136, 137, 138, 139],
    [143, 144, 145],
    [147, 148],
    [149, 150, 151]
]



def perform_merge(indices, all_content):
    new_list = []

    for i in indices:
        new_list+=all_content[i]
    
    return new_list

def merge():
    merged_data = []
    for indices in merge_indices:
        merged_data.append(perform_merge(indices, all_content))
        
    return merged_data


merged_data = merge()

merged_data[0]
        

['January 5 – In the 1950 Sverdlovsk plane crash, a Lisunov Li-2 crashed near Sverdlovsk (now Yekaterinburg), Soviet Union, killing all 19 on board.',
 'March 7 – Northwest Orient Airlines Flight 307, a Martin 2–0–2, crashed near Minneapolis–Saint Paul International Airport after hitting a flagpole during approach, killing all 13 on board and two on the ground.',
 'March 12 – The Llandow air disaster: An Airflight Avro 689 Tudor V stalled and crashed on approach to Llandow Aewrodrome due to improper loading, resulting in a center of gravity exceeding the aft limit; 80 out of the 83 people on board die, at the time the worst air disaster in history.',
 'June 24 – Northwest Orient Airlines Flight 2501, a Douglas DC-4, with 58 people on board, disappeared without a trace over Lake Michigan.',
 'June 26 – Australian National Airways Amana, a Douglas DC-4, crashed after takeoff from Perth Airport, killing all 29 people on board.',
 'July 18 - Panair do Brasil Flight 099, a Lockheed L-049 Co

In [7]:
len(all_content), len(merged_data)

(153, 30)

In [8]:
new_data = []

In [9]:
i = 0
for content in all_content:
    if i<len(merge_indices) and all_content.index(content)==merge_indices[i][0]:
        all_content[merge_indices[i][0]] = merged_data[i]
        i+=1
len(all_content)

153

In [10]:
ind = 0
i = 0

while i<len(all_content):
    if ind<len(merge_indices) and i==merge_indices[ind][0]:
        new_data.append(all_content[merge_indices[ind][0]])
        i = merge_indices[ind][-1]
        ind+=1
    else:    
        new_data.append(all_content[i])
    i+=1

len(new_data)

109

In [11]:
merge_indices = [
    [43, 44],
    [86, 87],
    [88, 89],
    [102, 103]
]

def perform_merge(indices, all_content):
    new_list = []

    for i in indices:
        new_list+=all_content[i]
    
    return new_list

def merge():
    merged_data = []
    for indices in merge_indices:
        merged_data.append(perform_merge(indices, all_content))
        
    return merged_data


merged_data = merge()

merged_data[0]


['May 20 – Capital Airlines Flight 300, a Vickers Viscount, collided with a USAF T-33, killing all 13 on board the Viscount and one of two on board the T-33; the T-33s pilot parachuted to the ground and survived.',
 'May 25 – An Avro York 685 cargo aircraft crashed during a forced landing after an engine catches fire en route from Karachi to Delhi, killing four of the five people on board.',
 'June 2 - Aeronaves de México Flight 111, a Lockheed Constellation, hits a mountain shortly after takeoff from Guadalajara, killing all 46 on board.',
 'August 9 – Central African Airways Flight 890, a Vickers Viscount, crashes due to pilot error near Benina International Airport, Libya.  Of the 54 on board, 36 are killed.',
 'August 14 – KLM Flight 607-E, a Lockheed L-1049 Super Constellation (named Hugo de Groot) en route from Amsterdam to New York, crashed in the Atlantic Ocean shortly after takeoff from Shannon Airport in Ireland, killing all 99 passengers and crew, including six members of th

In [12]:
i = 0
for content in new_data:
    if i<len(merge_indices) and new_data.index(content)==merge_indices[i][0]:
        new_data[merge_indices[i][0]] = merged_data[i]
        i+=1
print(len(new_data))

109


In [13]:
ind = 0
i = 0

new_new_data = []

while i<len(new_data):
    if ind<len(merge_indices) and i==merge_indices[ind][0]:
        new_new_data.append(new_data[merge_indices[ind][0]])
        i = merge_indices[ind][-1]
        ind+=1
    else:    
        new_new_data.append(new_data[i])
    i+=1

len(new_new_data)

105

In [14]:
final_data = {}

for year, data in zip(all_years, new_new_data):
    final_data[year] = data


In [15]:
print(final_data['1919'][1])

August 2 – A Caproni Ca.48 crashed at Verona, Italy, during a flight from Venice to Taliedo, Milan, killing all on board (14, 15, or 17 people, according to different sources).


In [16]:
# import spacy

# nlp = spacy.load('en_core_web_sm')
# text = final_data['1919'][0]

# doc = nlp(text)

# airlines = [str(ent) for ent in doc.ents if ent.label_=='ORG']
# print(airlines)

# airline_words = ["Air", "Jet", "Fly", "Sky", "Wings", "Express", "Global", "International", "World", "Aviation", "Airways", "Cargo",
#     "Aero", "Travel", "Pacific", "Charter", "National", "Royal"]

# for airline in airlines:
#     for keyword in airline_words:
#         if keyword in airline.split(" "):
#             print(airline)

In [17]:
len(final_data['1973'])

36

In [18]:
def load_json():
    import json
    with open('airline_by_year.json', 'r') as f:
        airline_by_year = json.load(f)

    return airline_by_year

airline_by_year = load_json()

In [19]:
# missed_years = []
# for year, incidents in final_data.items():
#     if len(incidents) != len(airline_by_year[year]):
#         missed_years.append(year)

In [20]:
airplane_set = set()
for year, incidents in airline_by_year.items():
    temp_set = set(incidents)
    for airline in temp_set:
        try:
            airplane_set.add(airline)
        except:
            print("Already in set")
    

In [21]:
# airplane_set

## Create Train Data for Custom NER

In [22]:
import re

TRAIN_DATA = []

for year, incidents in final_data.items():
    for incident in incidents:
        entities = []
        for airline in airplane_set:
            matches = [(m.start(), m.end()) for m in re.finditer(rf'\b{re.escape(airline)}\b', incident)]
            for start_index, end_index in matches:
                entities.append((start_index, end_index, "AIRLINE"))

        if entities:  # Only add if at least one airline is found
            TRAIN_DATA.append((incident, {'entities': entities}))


### Remove overlapping entities

In [23]:
def remove_overlapping_entities(entities):
    entities = sorted(entities, key=lambda x: (x[0], x[1]))  # Sort by start index
    filtered_entities = []
    
    prev_start, prev_end, prev_label = entities[0]
    filtered_entities.append((prev_start, prev_end, prev_label))

    for start, end, label in entities[1:]:
        if start < prev_end:  # Overlapping entity found
            continue  # Skip the smaller entity
        filtered_entities.append((start, end, label))
        prev_start, prev_end, prev_label = start, end, label
    
    return filtered_entities

# Apply this function before training
TRAIN_DATA_FIXED = [(text, {"entities": remove_overlapping_entities(annotations["entities"])}) for text, annotations in TRAIN_DATA]

## Training a custom NER pipeline

In [ ]:
import spacy
# from spacy.training.example import Example

# def train_ner():
    # nlp = spacy.blank("en") # create a blank spacy model

    # if 'ner' not in nlp.pipe_names:
    #     ner = nlp.create_pipe("ner") # create a new component
    #     nlp.add_pipe("ner", last=True) # add it to the pipeline if not present
    # else:
    #     ner = nlp.get_pipe("ner") # get the component from the pipeline if it exists

    # iterate over the TRAIN_DATA and get the entities
    # for _, annotations in TRAIN_DATA_FIXED: 
    #     for ent in annotations.get("entities"):
    #         ner.add_label(ent[2])

    # optimizer = nlp.begin_training() # initialize the optimizer to update the model

"""
    Runs 30 training epochs.
    Converts each text-annotation pair into a spaCy Example object.
    Updates the NER model with the training examples.
    Uses a dropout rate of 50% (drop=0.5) to prevent overfitting.
"""
    # for epoch in range(30):
    #     print(f"Epoch number - {epoch+1}")
    #     for text, annotations in TRAIN_DATA_FIXED:
    #         # print(text)
    #         example = Example.from_dict(nlp.make_doc(text), annotations)
    #         nlp.update([example], drop=0.5)

    # nlp.to_disk("airline_ner") # save to disk

In [25]:
airline_ner = spacy.load("airline_ner")

# Test the trained NER model
doc = airline_ner("Aeroflot Flight 964, a Tupolev Tu-104, crashed while on approach to Domodedovo International Airport, Moscow, Russia after the pilots became disorientated following an electrical failure, killing all 122 on board in the deadliest accident involving the Tu-104.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Aeroflot AIRLINE


In [26]:
year_to_airline_mapping = {}

def extract_airline(incident):
    doc = airline_ner(incident)
    airlines = []
    for ent in doc.ents:
        airlines.append(ent.text)
    
    return airlines[0] if len(airlines)==1 else airlines

for year, incidents in final_data.items():
    airline_involved = []
    for incident in incidents:
        temp = extract_airline(incident)
        if type(temp)==list:
            for i in temp:
                airline_involved.append(i)
        else:
            airline_involved.append(temp)
    year_to_airline_mapping[year] = airline_involved


In [29]:
incidents_by_year = []
for crash in final_data.values():
    incident = []
    for j in crash:
        incident.append(j)
    incidents_by_year.append(incident)

In [30]:
# len(incidents_by_year)

## Transform

In [31]:
import spacy
import re
import pandas as pd

nlp = spacy.load("en_core_web_sm")

# Keywords for regex pattern
keywords = [
    "crashed at", "crashed near", "crashed while", "exploded in", "in the", "over the",
    "crash site", "crash", "in", "disappeared over", "over", "mid-air collision", "collision"
]

# Create regex pattern dynamically
pattern = r"(?:{})\s+([\w\s,]+?)(?:\.\s|,|\s+due|\s+after|\s+killing|\s+leading|$)".format("|".join(map(re.escape, keywords)))

def extract_crash_site(text):
    """Extract crash site using regex, NER, and POS patterns"""
    doc = nlp(text)

    # 1️⃣ Regex extraction
    regex_matches = re.findall(pattern, text)

    # 2️⃣ NER-based extraction
    ner_matches = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC", "FAC"]]

    # 3️⃣ POS-based pattern extraction
    pos_matches = []
    for i, token in enumerate(doc):
        if token.pos_ == "VERB" and i + 4 < len(doc):
            next_tokens = [doc[i + j] for j in range(1, 5)]
            if (next_tokens[0].pos_ == "ADP" and next_tokens[1].pos_ in ["NOUN", "PROPN"]
                    and next_tokens[2].pos_ == "ADP" and next_tokens[3].pos_ == "PROPN"):
                pos_matches.append(f"{next_tokens[1].text} {next_tokens[2].text} {next_tokens[3].text}")

    # Merge results, filter unique, and return most relevant one
    all_sites = list(set(regex_matches + ner_matches + pos_matches))
    
    if all_sites:
        return all_sites[0]  # Return first identified location
    return "NaN"  # No location found

# Process each year's incidents
yearly_results = []
start_year = 1919  # Modify based on the first year in your dataset

for i, incidents in enumerate(incidents_by_year):
    year = start_year + i  # Assigning years sequentially
    for incident in incidents:
        crash_site = extract_crash_site(incident)
        yearly_results.append({"Year": year, "Incident": incident, "Crash Site": crash_site})

# Convert results to DataFrame for better readability
df = pd.DataFrame(yearly_results)


In [32]:
def clean_crash_site(site):
    # Remove unnecessary phrases such as "International Airport", "Airport", "approach to", "to", "from", etc.
    site = re.sub(r"(International\sAirport|Airport|approach\sto|landing\sat|in\swhich|flight\srecorder|to|from|\s+)", "", site)

    # Remove any extra "the" at the beginning of the location name
    site = re.sub(r"^the\s", "", site)

    # Normalize common terms (optional, e.g., handle airport abbreviations)
    site = site.strip()

    return site

# Apply the cleaning function to the 'crash_site' column
df['cleaned_crash_site'] = df['Crash Site'].apply(clean_crash_site)

# Display the updated DataFrame
# df.head(20)

In [33]:
# df.shape

In [34]:
import numpy as np
def keep_gpe_loc(text):
    doc = nlp(text)  # Process the text with SpaCy
    for ent in doc.ents:  # Iterate over the entities in the text
        if ent.label_ in ['GPE', 'LOC']:  # Check if the entity is "GPE" or "LOC"
            return text  # Keep the entry if one of the entities is "GPE" or "LOC"
    return np.nan  # Remove the entry if no such entities are found

# Apply the filtering function to the 'crash_site' column
df['cleaned_crash_site'] = df['cleaned_crash_site'].apply(keep_gpe_loc)
# df.head(20), df.shape

In [35]:
# df[df['Year']==2023]

In [36]:
# len(year_to_airline_mapping)

In [37]:
all_incidents = []
all_airlines = []

for key, val in final_data.items():
    key = key.strip()
    all_incidents.append(val)
    all_airlines.append(year_to_airline_mapping[key])
    

In [40]:
all_months = []
for key, val in final_data.items():
    months = []
    for i in val:
        months.append(i.split(" ")[0])
    all_months.append(months)

len(all_months)

105

In [41]:
len(all_months), len(all_airlines)

(105, 105)

In [ ]:
data = []

for year in all_years:
    for month, incident, airline in zip(all_months, all_incidents, all_airlines):
        for i, j, k  in zip(month, incident,airline):
            data.append([year, i, j, k])

## Load

In [ ]:
df_final = pd.DataFrame(data, columns=['year', 'month', 'incident', 'airline/s_involved'])

df_final.head()



,year,month,incident,airline/s_involved
0,1919,July,July 21 – The Goodyear dirigible Wingfoot Air ...,Goodyear
1,1919,August,"August 2 – A Caproni Ca.48 crashed at Verona, ...",Caproni
2,1919,August,August 26 – A SNETA Farman Goliath ditched in ...,SNETA
3,1919,March,March 31 – A Beijing-Han Airlines Handley Page...,Beijing-Han Airlines
4,1919,January,January 13 – An Aeromarine 75 seaplane of Aero...,Aeromarine West Indies Airways


In [ ]:
df_final[['airline/s_involved']]

,airline/s_involved
0,Goodyear
1,Caproni
2,SNETA
3,Beijing-Han Airlines
4,Aeromarine West Indies Airways
...,...
187000,Air Busan
187001,Light Air Services
187002,American Eagle
187003,Bering Air


In [42]:
import pandas as pd

# Assuming all_data, all_months, and all_airlines are correctly structured
data_list = []

for idx, (year, incidents) in enumerate(final_data.items()):
    months = all_months[idx]  # Get the corresponding months list
    airlines = all_airlines[idx]  # Get the corresponding airlines list
    
    # Zip together month and airline for each incident in that year
    for month, airline in zip(months, airlines):
        data_list.append([year, month, airline])

# Create DataFrame
df = pd.DataFrame(data_list, columns=['year', 'month', 'airline'])



In [43]:
df.shape

(1781, 3)

In [44]:
df.head()

,year,month,airline
0,1919,July,Goodyear
1,1919,August,Caproni
2,1921,August,SNETA
3,1922,March,Beijing-Han Airlines
4,1923,January,Aeromarine West Indies Airways


In [48]:
import sqlite3

conn = sqlite3.connect('airline.db')
df.to_sql("airlines_table", conn, if_exists="replace", index=False) 

# Export SQL commands to a .sql file
with open("airlines.sql", "w", encoding="utf-8") as f:
    for line in conn.iterdump():  # Dumps SQL statements
        f.write(f"{line}\n")

conn.close()

In [50]:
df.to_csv("airline_crashes.csv", index=False)